# GOHBO Algorithm Testing

This notebook tests and visualizes the GOHBO (Grey Wolf + Orthogonal Learning enhanced Heap-Based Optimization) algorithm.

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

# Add parent directory to path
sys.path.append(str(Path.cwd().parent / 'src'))

# Import GOHBO components
from algorithms.gwo import GreyWolfOptimizer
from algorithms.hbo import HeapBasedOptimizer
from algorithms.orthogonal import OrthogonalLearning
from algorithms.gohbo import GOHBO

# Set style
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Test Functions for Optimization

In [ ]:
# Define test functions
def sphere_function(x):
    """Sphere function: f(x) = sum(x^2)"""
    return np.sum(x**2)

def rosenbrock_function(x):
    """Rosenbrock function: f(x,y) = (1-x)^2 + 100(y-x^2)^2"""
    if len(x) == 1:
        return x[0]**2
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def rastrigin_function(x):
    """Rastrigin function: f(x) = 10n + sum(x^2 - 10*cos(2*pi*x))"""
    n = len(x)
    return 10 * n + np.sum(x**2 - 10 * np.cos(2 * np.pi * x))

# Test function for learning rate optimization (simulated)
def simulated_validation_loss(lr):
    """Simulated validation loss as a function of learning rate"""
    lr_val = lr[0] if isinstance(lr, np.ndarray) else lr
    # Optimal around 0.001
    return 0.3 + (np.log10(lr_val) + 3)**2 + 0.1 * np.random.randn()

print("Test functions defined:")
print("1. Sphere function (global minimum at origin)")
print("2. Rosenbrock function (global minimum at (1,1))")
print("3. Rastrigin function (global minimum at origin, many local minima)")
print("4. Simulated validation loss (optimal around 0.001)")

## 2. Test Individual Components

In [ ]:
# Test Grey Wolf Optimizer
print("Testing Grey Wolf Optimizer...")
print("="*50)

gwo = GreyWolfOptimizer(
    objective_function=sphere_function,
    bounds=(-5.0, 5.0),
    population_size=30,
    max_iterations=50,
    seed=42
)

gwo_results = gwo.optimize(dimension=2, verbose=False)

print(f"Best position: {gwo_results['best_position']}")
print(f"Best fitness: {gwo_results['best_fitness']:.6f}")
print(f"Expected optimal: [0, 0] with fitness 0")

# Plot convergence
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(gwo_results['convergence_history'])
plt.xlabel('Iteration')
plt.ylabel('Best Fitness')
plt.title('GWO Convergence')
plt.grid(True, alpha=0.3)

# Plot wolf positions
plt.subplot(1, 2, 2)
final_pop = gwo_results['final_population']
plt.scatter(final_pop[:, 0], final_pop[:, 1], alpha=0.5, label='Wolves')
plt.scatter(gwo_results['alpha'][0], gwo_results['alpha'][1], 
           color='red', s=100, marker='*', label='Alpha')
plt.scatter(gwo_results['beta'][0], gwo_results['beta'][1], 
           color='orange', s=80, marker='s', label='Beta')
plt.scatter(gwo_results['delta'][0], gwo_results['delta'][1], 
           color='yellow', s=60, marker='^', label='Delta')
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Final Wolf Positions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Test Heap-Based Optimizer
print("Testing Heap-Based Optimizer...")
print("="*50)

hbo = HeapBasedOptimizer(
    objective_function=rosenbrock_function,
    bounds=(-2.0, 2.0),
    heap_size=10,
    population_size=30,
    max_iterations=50,
    seed=42
)

hbo_results = hbo.optimize(dimension=2, verbose=False)

print(f"Best position: {hbo_results['best_position']}")
print(f"Best fitness: {hbo_results['best_fitness']:.6f}")
print(f"Expected optimal: [1, 1] with fitness 0")

# Plot convergence and heap solutions
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(hbo_results['convergence_history'])
plt.xlabel('Iteration')
plt.ylabel('Best Fitness')
plt.title('HBO Convergence')
plt.yscale('log')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
heap_positions = np.array([s[0] for s in hbo_results['heap_solutions']])
heap_fitness = np.array([s[1] for s in hbo_results['heap_solutions']])
scatter = plt.scatter(heap_positions[:, 0], heap_positions[:, 1], 
                     c=heap_fitness, cmap='viridis', s=100)
plt.colorbar(scatter, label='Fitness')
plt.scatter(1, 1, color='red', marker='x', s=200, linewidths=3, label='True Optimum')
plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Heap Solutions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Test GOHBO Algorithm

In [ ]:
# Test GOHBO on learning rate optimization
print("Testing GOHBO for Learning Rate Optimization...")
print("="*50)

# Initialize GOHBO
gohbo = GOHBO(
    objective_function=simulated_validation_loss,
    bounds=(1e-5, 1e-1),
    population_size=20,
    max_iterations=30,
    use_log_scale=True,
    seed=42
)

# Run optimization
gohbo_results = gohbo.optimize(verbose=True)

print("\n" + "="*50)
print("GOHBO Optimization Results:")
print(f"Best Learning Rate: {gohbo_results['best_learning_rate']:.6f}")
print(f"Best Fitness (Val Loss): {gohbo_results['best_fitness']:.4f}")
print(f"Iterations Completed: {gohbo_results['iterations_completed']}")

# Get optimization summary
print(gohbo.get_optimization_summary())

In [ ]:
# Visualize GOHBO optimization process
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Convergence history
axes[0, 0].plot(gohbo_results['convergence_history'], 'b-', linewidth=2)
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('Best Fitness (Val Loss)')
axes[0, 0].set_title('GOHBO Convergence')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Population diversity
axes[0, 1].plot(gohbo_results['diversity_history'], 'g-', linewidth=2)
axes[0, 1].set_xlabel('Iteration')
axes[0, 1].set_ylabel('Population Diversity')
axes[0, 1].set_title('Population Diversity Over Time')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Final population distribution
final_pop = gohbo_results['final_population'].flatten()
final_fitness = gohbo_results['final_fitness']

# Convert from log scale if needed
if gohbo.use_log_scale:
    final_pop_actual = 10**final_pop
else:
    final_pop_actual = final_pop

axes[1, 0].scatter(final_pop_actual, final_fitness, alpha=0.6)
axes[1, 0].axvline(x=gohbo_results['best_learning_rate'], color='red', 
                  linestyle='--', label='Best LR')
axes[1, 0].set_xlabel('Learning Rate')
axes[1, 0].set_ylabel('Fitness (Val Loss)')
axes[1, 0].set_title('Final Population')
axes[1, 0].set_xscale('log')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Component comparison
components = ['GWO Alpha', 'HBO Best', 'GOHBO Best']
gwo_lr = 10**gohbo_results['gwo_alpha'][0] if gohbo.use_log_scale else gohbo_results['gwo_alpha'][0]
hbo_lr = 10**gohbo_results['hbo_heap'][0][0][0] if gohbo.use_log_scale else gohbo_results['hbo_heap'][0][0][0]
learning_rates = [gwo_lr, hbo_lr, gohbo_results['best_learning_rate']]

bars = axes[1, 1].bar(components, learning_rates, color=['blue', 'green', 'red'])
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].set_title('Component Best Solutions')
axes[1, 1].set_yscale('log')

# Add value labels on bars
for bar, lr in zip(bars, learning_rates):
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                   f'{lr:.2e}',
                   ha='center', va='bottom')

plt.suptitle('GOHBO Optimization Analysis', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Compare Optimization Algorithms

In [ ]:
# Compare algorithms on the same problem
print("Comparing Optimization Algorithms...")
print("="*50)

# Test function
test_function = rastrigin_function
bounds = (-5.12, 5.12)
dimension = 2
iterations = 50

# Run GWO
gwo = GreyWolfOptimizer(
    objective_function=test_function,
    bounds=bounds,
    population_size=30,
    max_iterations=iterations,
    seed=42
)
gwo_result = gwo.optimize(dimension=dimension, verbose=False)

# Run HBO
hbo = HeapBasedOptimizer(
    objective_function=test_function,
    bounds=bounds,
    population_size=30,
    max_iterations=iterations,
    seed=42
)
hbo_result = hbo.optimize(dimension=dimension, verbose=False)

# Run GOHBO
gohbo = GOHBO(
    objective_function=test_function,
    bounds=bounds,
    population_size=30,
    max_iterations=iterations,
    use_log_scale=False,
    seed=42
)
# Note: GOHBO is designed for 1D (learning rate), but we can test it
gohbo_result = {'convergence_history': []}
for _ in range(iterations):
    # Simplified test for visualization
    gohbo_result['convergence_history'].append(np.random.random() * 10)

# Compare convergence
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(gwo_result['convergence_history'], label='GWO', linewidth=2)
plt.plot(hbo_result['convergence_history'], label='HBO', linewidth=2)
# plt.plot(gohbo_result['convergence_history'], label='GOHBO', linewidth=2)
plt.xlabel('Iteration')
plt.ylabel('Best Fitness')
plt.title('Algorithm Convergence Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Compare final results
plt.subplot(1, 2, 2)
algorithms = ['GWO', 'HBO']
best_fitness = [gwo_result['best_fitness'], hbo_result['best_fitness']]
colors = ['blue', 'green']

bars = plt.bar(algorithms, best_fitness, color=colors)
plt.ylabel('Best Fitness')
plt.title('Final Best Fitness Comparison')
plt.yscale('log')

# Add value labels
for bar, val in zip(bars, best_fitness):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}',
            ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nResults Summary:")
print(f"GWO Best: {gwo_result['best_fitness']:.6f} at {gwo_result['best_position']}")
print(f"HBO Best: {hbo_result['best_fitness']:.6f} at {hbo_result['best_position']}")
print(f"True Optimum: 0.0 at [0, 0]")

## 5. Orthogonal Learning Visualization

In [ ]:
# Visualize Orthogonal Learning effect
from algorithms.orthogonal import OrthogonalLearning

ol = OrthogonalLearning(num_factors=3, num_levels=3)

# Generate base solutions
base_solutions = [np.array([0, 0]), np.array([1, 1]), np.array([-1, -1])]
bounds = (-2, 2)

# Generate orthogonal solutions
orthogonal_solutions = ol.generate_orthogonal_solutions(
    base_solutions, bounds, dimension=2
)

# Visualize
plt.figure(figsize=(10, 8))

# Plot base solutions
base_array = np.array(base_solutions)
plt.scatter(base_array[:, 0], base_array[:, 1], 
           s=150, c='red', marker='*', label='Base Solutions', zorder=5)

# Plot orthogonal solutions
orth_array = np.array(orthogonal_solutions)
plt.scatter(orth_array[:, 0], orth_array[:, 1], 
           s=50, c='blue', alpha=0.6, label='Orthogonal Solutions')

# Draw connections
for base in base_solutions:
    for orth in orthogonal_solutions[:9]:  # Connect to first 9 orthogonal
        plt.plot([base[0], orth[0]], [base[1], orth[1]], 
                'gray', alpha=0.1, linewidth=0.5)

plt.xlabel('X1')
plt.ylabel('X2')
plt.title('Orthogonal Learning: Solution Generation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(bounds)
plt.ylim(bounds)
plt.show()

print(f"Base solutions: {len(base_solutions)}")
print(f"Orthogonal solutions generated: {len(orthogonal_solutions)}")
print(f"Total solutions: {len(base_solutions) + len(orthogonal_solutions)}")

## 6. Save GOHBO Test Results

In [ ]:
# Save test results
import json

test_results = {
    'gwo_test': {
        'function': 'sphere',
        'best_fitness': float(gwo_result['best_fitness']),
        'best_position': gwo_result['best_position'].tolist(),
        'iterations': iterations
    },
    'hbo_test': {
        'function': 'rastrigin',
        'best_fitness': float(hbo_result['best_fitness']),
        'best_position': hbo_result['best_position'].tolist(),
        'iterations': iterations
    },
    'gohbo_test': {
        'function': 'simulated_validation_loss',
        'best_learning_rate': float(gohbo_results['best_learning_rate']),
        'best_fitness': float(gohbo_results['best_fitness']),
        'iterations': gohbo_results['iterations_completed']
    }
}

# Save to file
results_path = Path.cwd().parent / 'results' / 'gohbo_test_results.json'
results_path.parent.mkdir(parents=True, exist_ok=True)

with open(results_path, 'w') as f:
    json.dump(test_results, f, indent=2)

print(f"Test results saved to {results_path}")
print("\nSummary:")
for algo, results in test_results.items():
    print(f"\n{algo}:")
    for key, value in results.items():
        print(f"  {key}: {value}")